# Tarea #1 Ciencia de Datos 2 Santiago Henao Rendón 
## Pipeline de Procesamiento de Datos (Titanic)

El objetivo de este trabajo es seleccionar un dataset, realizar un mini-EDA (Análisis Exploratorio de Datos) e implementar un Pipeline de Preprocesamiento robusto con scikit-learn.

### Paso 1: Cargar librerías necesarias para el ejercicio.
Comenzamos importando las librerías necesarias de Python (pandas para manejo de datos, y varios módulos de scikit-learn para el pipeline y el modelado).

Seleccionamos un subconjunto de 7 variables del dataset para cumplir con el requisito de tener entre 5 y 10 variables.

In [3]:
# Importación de librerías esenciales
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression

# 1. Cargar el dataset 'train.csv'
df_completo = pd.read_csv('train.csv')

# 2. Seleccionar el subconjunto de columnas (Requisito: 5 a 10 variables)
COLUMNAS_USAR = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked']
df = df_completo[COLUMNAS_USAR].copy()

# Separar variables predictoras (X) y objetivo (y)
X = df.drop('Survived', axis=1)
y = df['Survived']

# Dividir para entrenamiento y prueba (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dimensiones del set de entrenamiento (X_train): {X_train.shape}")

Dimensiones del set de entrenamiento (X_train): (712, 6)


### Paso 2: Mini EDA y Definición de Estrategias de Procesamiento
Realizamos una breve exploración del set de entrenamiento para identificar los tipos de variables, datos faltantes y definir las estrategias de preprocesamiento.
#### 2.1: Estadísticas Descriptivas e Identificación de Datos Faltantes (N/A)
Verificamos la cantidad de valores nulos por columna para determinar la estrategia de imputación.


In [7]:
print("--- Estadísticas Descriptivas de Variables Numéricas en X_train ---")
print(X_train[['Age', 'SibSp', 'Fare']].describe())

print("\n--- Frecuencia de Categorías y Faltantes en X_train ---")
for col in ['Pclass', 'Sex', 'Embarked']:
    print(f"\n--- Columna: {col} ---")
    print(X_train[col].value_counts(dropna=False)) # dropna=False incluye los NaNs en el conteo



--- Estadísticas Descriptivas de Variables Numéricas en X_train ---
              Age       SibSp        Fare
count  572.000000  712.000000  712.000000
mean    29.498846    0.553371   32.586276
std     14.500059    1.176404   51.969529
min      0.420000    0.000000    0.000000
25%     21.000000    0.000000    7.925000
50%     28.000000    0.000000   14.454200
75%     38.000000    1.000000   30.500000
max     80.000000    8.000000  512.329200

--- Frecuencia de Categorías y Faltantes en X_train ---

--- Columna: Pclass ---
Pclass
3    398
1    163
2    151
Name: count, dtype: int64

--- Columna: Sex ---
Sex
male      467
female    245
Name: count, dtype: int64

--- Columna: Embarked ---
Embarked
S      525
C      125
Q       60
NaN      2
Name: count, dtype: int64


In [12]:
X_train['Pclass'] = X_train['Pclass'].astype(str)
X_test['Pclass'] = X_test['Pclass'].astype(str)

# Y ajusta la definición de categories en el Pipeline:
ordinal_categories = [['3', '2', '1']] 
# Ahora son strings.

print("--- Conteo de Datos Faltantes (NaN) en X_train ---")
print(X_train.isnull().sum())
print("\n--- Tipos de Datos de las Variables ---")
print(X_train.dtypes)

--- Conteo de Datos Faltantes (NaN) en X_train ---
Pclass        0
Sex           0
Age         140
SibSp         0
Fare          0
Embarked      2
dtype: int64

--- Tipos de Datos de las Variables ---
Pclass       object
Sex          object
Age         float64
SibSp         int64
Fare        float64
Embarked     object
dtype: object


#### Hallazgos Clave:
- La variable Age contiene 140 datos faltantes, permitiendo adoptar una estrategia de imputación por la mediana.
- La variable categórica de Embarked contiene 2 datos faltantes. Se adopta estrategia de imputación por la Moda.

#### 2.2 Resumen de Estrategias de Preprocesamiento

Esta tabla resume la identificación de datos faltantes, la estrategia para llenar esos vacíos (**Imputación**), y la estrategia para normalizar el rango de valores (**Escalamiento/Codificación**) para todas las variables utilizadas en el pipeline.

| Columna | Tipo de Variable | Datos Faltantes | Estrategia de Imputación | Estrategia de Codificación / Escalamiento | Razón de la Estrategia |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **`Age`** | Numérica (Continua) | **147** | Mediana | **MinMaxScaler** | Imputación con **Mediana** por ser robusta a *outliers*. **MinMaxScaler** para normalizar el rango entre $[0, 1]$. |
| **`SibSp`** | Numérica (Conteo) | 0 | N/A | **StandardScaler** | **StandardScaler** para estandarizar el set de datos ($\mu=0, \sigma=1$), útil para variables de conteo. |
| **`Fare`** | Numérica (Continua) | 0 | N/A | **StandardScaler** | Estandariza la alta varianza y los valores extremos (outliers) de la tarifa. |
| **`Sex`** | Categórica (Nominal) | 0 | N/A | **OneHotEncoder** | Variable sin orden, crea columnas binarias para evitar que el modelo interprete la etiqueta como magnitud. |
| **`Embarked`** | Categórica (Nominal) | **2** | Moda | **OneHotEncoder** | Imputación por la **Moda** debido a la baja cantidad de nulos. Se codifica para convertir texto a formato numérico. |
| **`Pclass`** | Categórica (Ordinal) | 0 | N/A | **OrdinalEncoder** | Variable con orden (1ª > 2ª > 3ª clase). Se codifica con valores numéricos que respetan esta jerarquía. |

### Paso 3: Diseño e Implementación del Pipeline
Construimos los pipelines individuales (secuencias de Imputación y Escalamiento/Codificación) y los combinamos en un solo ColumnTransformer.

#### 3.1 Creación de Componentes Individuales:

In [ ]:
# 1. 🌲 Pipeline para Numéricas (StandardScaler)
num_std_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 2. 🌲 Pipeline para Age (MinMaxScaler)
age_minmax_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

# 3. 🏷️ Pipeline para Nominales (OneHotEncoder)
nominal_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 4. 📐 Pipeline para Ordinales (OrdinalEncoder)

# Definición del orden: De la peor clase ('3') a la mejor ('1')
# NOTA: Deben ser strings, ya que convertimos la columna.
# Es una lista de listas, donde cada lista interior es la columna.

ordinal_categories = [['3', '2', '1']] 


ordinal_pipeline = OrdinalEncoder(categories=ordinal_categories) 



#### 3.2: Combinación con ColumnTransformer
El ColumnTransformer dirige cada pipeline de transformación al grupo de columnas correspondiente.

In [15]:
preprocessor = ColumnTransformer(
    transformers=[
        # Aplica StandardScaler a SibSp y Fare
        ('num_std', num_std_pipeline, ['SibSp', 'Fare']),
        # Aplica MinMaxScaler a Age
        ('age_minmax', age_minmax_pipeline, ['Age']),
        # Aplica OneHotEncoder a Sex y Embarked
        ('cat_nom', nominal_pipeline, ['Sex', 'Embarked']),
        # Aplica OrdinalEncoder a Pclass
        ('cat_ord', ordinal_pipeline, ['Pclass'])
    ],
    remainder='drop' # Ignora cualquier columna que no haya sido especificada.
)

print("ColumnTransformer definido.")

ColumnTransformer definido.


#### 3.3 Pipeline Completo y Entrenamiento
El Pipeline final encadena el preprocesamiento con un modelo de Regresión Logística y entrena el modelo de forma eficiente.

In [18]:
# Pipeline completo: Preprocesador -> Clasificador
full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, solver='liblinear'))
])

# 🚀 Entrenar el Pipeline
full_pipeline.fit(X_train, y_train)

# Evaluación
score = full_pipeline.score(X_test, y_test)

print("\n----------------------------------------------------")
print("✅ Pipeline de Preprocesamiento y Modelo entrenado.")
print(f"Precisión del modelo en el set de prueba: {score:.4f}")
print("----------------------------------------------------")


----------------------------------------------------
✅ Pipeline de Preprocesamiento y Modelo entrenado.
Precisión del modelo en el set de prueba: 0.7989
----------------------------------------------------
